# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()

import module as mod     # functions are include in module.py

In [2]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [4]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [5]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [24]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())



Hombre = deaths['cause_group'].unique()

Hombre


array(['Multiple causes', 'Single cause'], dtype=object)

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [56]:
# Example 1
'''
dataset = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'Edad', ['Todas las edades'])
dataset.head()
'''

#dataset1 = mod.row_filter(deaths, 'cause_group', ['Single cause'])
#dataset2 = mod.row_filter(dataset1, 'Edad', ['Todas las edades'])
#dataset3 =mod.row_filter(dataset2, 'Sexo', ['Total'])
dataset4 = mod.row_filter(deaths, 'cause_code', ['059'])
dataset4

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,059 Enfermedades cerebrovasculares,Total,Todas las edades,1981,49000,059,Single cause,Enfermedades cerebrovasculares
1,059 Enfermedades cerebrovasculares,Total,Todas las edades,1983,48331,059,Single cause,Enfermedades cerebrovasculares
2,059 Enfermedades cerebrovasculares,Total,Todas las edades,1984,47699,059,Single cause,Enfermedades cerebrovasculares
3,059 Enfermedades cerebrovasculares,Total,Todas las edades,1985,47684,059,Single cause,Enfermedades cerebrovasculares
4,059 Enfermedades cerebrovasculares,Total,Todas las edades,1980,47475,059,Single cause,Enfermedades cerebrovasculares
...,...,...,...,...,...,...,...,...
2569,059 Enfermedades cerebrovasculares,Mujeres,Menos de 1 año,2014,0,059,Single cause,Enfermedades cerebrovasculares
2570,059 Enfermedades cerebrovasculares,Mujeres,De 10 a 14 años,2005,0,059,Single cause,Enfermedades cerebrovasculares
2571,059 Enfermedades cerebrovasculares,Total,De 5 a 9 años,2017,0,059,Single cause,Enfermedades cerebrovasculares
2572,059 Enfermedades cerebrovasculares,Mujeres,Menos de 1 año,2008,0,059,Single cause,Enfermedades cerebrovasculares


In [76]:
#dataset1 = mod.row_filter(deaths, 'cause_code', ['059'])
#dataset2 = mod.row_filter(dataset1, 'Sexo', ['Total'])
#dataset3 = mod.row_filter(dataset1, 'Sexo', ['Total'])

In [77]:
# Example 2
'''
group = ['cause_code','Periodo']
dataset = mod.groupby_sum(deaths, group)
dataset.head()
'''

group = ['Sexo','Periodo']
dataset = mod.groupby_sum(dataset4, group)
dataset

,Sexo,Periodo,Total
0,Total,1981,98000
1,Total,1983,96662
2,Total,1984,95398
3,Total,1985,95368
4,Total,1980,94950
...,...,...,...
112,Hombres,2013,23186
113,Hombres,2014,23146
114,Hombres,2016,23112
115,Hombres,2017,23110


In [78]:
# Example 3
'''
dataset = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset.head()
'''

dataset1 = mod.pivot_table(dataset, 'Sexo', 'Periodo')
dataset1

Sexo,Periodo,Hombres,Mujeres,Total
0,1980,40608,54342,94950
1,1981,41682,56318,98000
2,1982,39030,53246,92276
3,1983,40658,56004,96662
4,1984,40274,55124,95398
5,1985,40330,55038,95368
6,1986,37804,53796,91600
7,1987,36760,52164,88924
8,1988,37108,52202,89310
9,1989,36448,51304,87752


## ...and finally, show me some insights with Plotly!!!

In [80]:
# Cufflinks histogram
'''
dataset_column.iplot(kind='hist',
                     title='VIZ TITLE',
                     yTitle='AXIS TITLE',
                     xTitle='AXIS TITLE')
'''
dataset1.iplot(kind='line',
                     title='Enfermedades Cerebrovasculares',
                     yTitle='Numeros',
                     x ='Periodo',
                     y =['Total', 'Hombres', 'Mujeres'],
                     xTitle='Periodo')


In [ ]:
# Cufflinks bar plot
'''
dataset_bar.iplot(kind='bar',
                  x='VARIABLE',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')
'''

In [ ]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

In [ ]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''